## Task 1: 
Review Chapter 6 of the Geron book, complete problems 7 and 8, and create a Jupyter notebook (HW5a.ipynb) with your notes and explanations.
<br><br>

# Chapter 6: Decision Trees

### Training and Visualizing a Decision Tree

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

# train tree on the iris dataset
iris = load_iris(as_frame=True)
X_iris = iris.data[["petal length (cm)", "petal width (cm)"]].values
y_iris = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_iris, y_iris)

DecisionTreeClassifier(max_depth=2, random_state=42)

In [11]:
from sklearn.tree import export_graphviz

# visualize the trained tree
export_graphviz(
        tree_clf,
        out_file="iris_tree.dot",
        feature_names=["petal length (cm)", "petal width (cm)"],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

In [12]:
# load and display file
from graphviz import Source # pip install graphviz!

Source.from_file("iris_tree.dot")

ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

### Making Predictions

Travel down the tree until you reach a leaf node. It is similar to tree traversal in other tree data structures.

Each node has some attributes (instance variables):
* ```samples```: the number of training instances the node applies to.
* ```value```: the number of training instances of each class that the node applies to.
* ```gini```: measures *Gini impurity* (a representation of whether/how many nodes belong to the same class.

<br>

**Eq 6-1. Gini impurity**

$$
G_1 = 1 - \sum_{k=1}^n p_{i,k}^2
$$

* $G_i$ is the Gini impurity of the $i^{th}$ node.
* $p_{i,k}^2$ is the ratio of class $k$ instances among the training instances in the $i^{th}$ node.

### Estimating Class Probabilities

A decision tree can estimate the probability that an instance belongs to a particular class $k$. First the finds the leaf node for the instance in question, and then it returns the ratio of training instances of class $k$ in the leaf node. Output the probability that the instance belongs to each class with ```predict_proba()```. The ```predict()``` method outputs the class with the highest probability.

### The CART Training Algorithm

The *Classification and Regression Tree (CART)* algorithm is used to train decision trees. The algorithm splits the training set into two subsets using a single feature $k$ and a threshold. It searches for the pair $(k, t_K)$ that produces the purest subsets, weighted by their size.

$$
j(k,t_k) = \frac{m_{left}}{m}G_{left} + \frac{m_{right}}G_{right}
$$
* $G_{left/right}$ measures the impurity of the left/right subset.
* $m_{left/right}$ is the number of instances in the left/right subset.

CART recurses until it reaches ```max_depth``` or if it cannot find a split that will reduct impurity. Some other hyper params that control additional stopping conditions are: ```min_samples_split```, ```min_samples_leaf```, ```min_weight_fraction_leaf```, and ```max_leaf_nodes```.

### Computational Complexity

Traversing the tree is usually $O\log_2(m)$. The training algorithm compares all features (or up to ```max_features```) on all samples at each node.

### Gini Impurity vs Entropy

```DecisionTreeClassifier``` uses Gini impurity measure by default, but you can select the entropy impurity measure with ```criterion="entropy"```. The entropy of a set is zero when it contains instances of only one class.

**Eq 6-3. Entropy of the $i^{th}$ node**

$$
H_i = - \sum_{k=1\\{p_{i,k}\neq0}}^n p_{i,k} \log_2(p_{i,k})
$$

Choosing between Gini impurity and entropy does not make a big difference most of the time. Gini is a good default since it is slightly faster, but they lead to similar trees most of the time. When they differ, however, Gini tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly most balanced trees.

### Regularization Hyperparameters

Decision trees make few assumptions about the training data. If left unconstrained, the tree structure will likely overfit the data. This is a *nonparametric model* because the number of parameters ws not determined prior to training. In contrast, a *parametric model* (e.g., linear regression) has a predetermined number of parameters, so it's DoF are limited, but there is less risk of overfitting.

Regularization generally hyper params depend on the algorithm used, but you can usually restrict the ```max_depth```.

The ```DecisionTreeClassifier``` class has a few other parameters that similarly restrict the shape of the decision tree:

* ```max_features```: Maximum number of features that are evaluated for splitting at each node


* ```max_leaf_nodes```: Maximum number of leaf nodes


* ```min_samples_split```: Minimum number of samples a node must have before it can be split


* ```min_samples_leaf```: Minimum number of samples a leaf node must have to be created


* ```min_weight_fraction_leaf```: Same as min_samples_leaf but expressed as a fraction of the total number of weighted instances

### Regression

Decision trees can also perform regresson tasks using ```DecisionTreeRegressor```.

In [13]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

# generate a noisy quadratic dataset
np.random.seed(42)
X_quad = np.random.rand(200, 1) - 0.5  # a single random input feature
y_quad = X_quad ** 2 + 0.025 * np.random.randn(200, 1)

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X_quad, y_quad)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [16]:
# visualize the tree with graphviz
export_graphviz(
    tree_reg,
    out_file=str("regression_tree.dot"),
    feature_names=["x1"],
    rounded=True,
    filled=True
)

Source.from_file("regression_tree.dot")


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

Instead of predicting a class in each node, the decision tree predicts a value. This prediction is the average target value of all the training instances associated with the leaf node in question. Traversing is like traversing a binary search tree.

The CART algorithm works by splitting the training set in a way that minimizes the MSE.

**Eq 6-4. CART cost function for regression**

$$
J(k,t_k) = \frac{m_{\text{left}}}{m} MSE_{\text{left}} + \frac{m_{\text{right}}}{m} \text{MSE}_{\text{right}} \text{   where} 
\begin{cases} 
\text{MSE}_{node} = \frac{\sum_{i \in node} (\hat y_{node}-y^{(i)})^2}{m_{node}} \\
\hat y_{node} = \frac{\sum_{i \in node} y^{(i)})^2}{m_{node}}
\end{cases}
$$

Without regularization, decision trees are also prone to overfitting regression tasks.

### Sensitivity to Axis Orientation

Decision trees like to make orthogonal decision boundaries (see images in the textbook for this chapter), so they are sensitive to data orientation. We can limit this problem by scaling the data and then applying PCA.

In [17]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


# small pipeline that scales and rotates data using PCA
pca_pipeline = make_pipeline(StandardScaler(), PCA())
X_iris_rotated = pca_pipeline.fit_transform(X_iris)
tree_clf_pca = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf_pca.fit(X_iris_rotated, y_iris)

DecisionTreeClassifier(max_depth=2, random_state=42)

In [18]:
# vizualize with graphviz

### Decision Trees have a High Varience

This is the main issue with decision trees. Small hyper param changes or data may produce very different models. Also, SKL's tree training algorithm chooses features stochastically, so if ```random_state``` is not set, even the same data can produce different trees.

We can reduce variance by averaging predictions over many trees. This *ensemble* of trees is called a *random forest*, and it is a very powerful model.

# Problem 7.
Train and fine-tune a decision tree for the moons dataset by following these steps:
1. Use ```make_moons(n_samples=10000, noise=0.4)``` to generate a moons dataset.


2. Use ```train_test_split()``` to split the dataset into a training set and a test set.


3. Use grid search with cross-validation (with the help of the ```GridSearchCV``` class) to find good hyperparameter values for a ```DecisionTreeClassifier```. Hint: try various values for max_leaf_nodes.


4. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

In [25]:
np.random.seed(42)

In [26]:
# step 1

from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=10000, noise=0.4)

In [27]:
# step 2

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons,
                                                    test_size=0.2)

In [34]:
# step 3

from sklearn.model_selection import GridSearchCV

params = { # idk how to come up with these range though...
    'max_leaf_nodes': list(range(2, 100)),
    'max_depth': list(range(1, 7)),
    'min_samples_split': [2, 3, 4]
}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(),
                              params,cv=3)

grid_search_cv.fit(X_train, y_train)
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_depth=6, max_leaf_nodes=14)

In [29]:
# step 4

from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.855

# Problem 8.

Grow a forest using the following steps:

1. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s ```ShuffleSplit``` class for this.


2. Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy.


3. Now comes the magic. For each test set instance, generate the predictions of the 1,000 decision trees, and keep only the most frequent prediction (you can use SciPy’s ```mode()``` function for this). This approach gives you majority-vote predictions over the test set.


4. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a random forest classifier!

In [30]:
# step 1

from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

# ShuffleSplit yields indices to split
rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances)

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [31]:
# step 2

from sklearn.base import clone

# clone the decision tree with the optimized hyper params (from problem 7)
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

# fit each individual tree
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# average their scores
np.mean(accuracy_scores)

0.7989809999999999

In [32]:
# step 3

Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)


from scipy.stats import mode

# choose the most common prediction for each instance across trees
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [33]:
# step 4

accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8615